# **Emotion Detection**

### **Introduction**

**Emode** is an emotion detection application that is written in Jupyter notebook using Python and various libraries.

Emotion detection is the process of detecting a person's emotional state. Some of the common ways to detect emotion is by looking into facial features, voice signals etc.

In this project, I'll be implementing three different types of emotion detection and finding a common pattern for a given input. The three types used here are:

- Facial Recognition (using DeepFace)
- Speech Recognition (using Librosa)
- Text Recognition (using Neattext)

### **Installing and Importing Libraries**

In this project, we are going to use the following libraries:



In [1]:
# Install the necessary libraries.
# !pip install opencv-python

In [2]:
# Import the necessary libraries.
import os
import shutil
import datetime
import csv
from concurrent.futures import ThreadPoolExecutor

# Import visual detection libraries.
import cv2 as cv
from deepface import DeepFace
import tensorflow as tf

In [3]:
# Suppress most TensorFlow logs (0 = all logs, 2 = warnings and errors only)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Suppress TensorFlow info and warnings (but not errors)
tf.get_logger().setLevel('ERROR')

In [4]:
# Ensure Tensorflow is using the GPU
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try: 
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as re:
        print(f"RuntimeError: {re}")

### **Creating the folders required for the project**

First, let's create the necessary folders so that we can organize and store different sets of files into different folders for processing.

In [5]:
# Creating a list of all the required additional folders.
folders = [
    "../logs", "../model",
    "../media", "../media/audio", "../media/audio-chunks", "../media/video", "../media/documents"
]

# Function to create folders.
def create_and_clean_folders(folder):
    print(os.getcwd())
    if not os.path.exists(folder):
        os.makedirs(folder)
    elif folder in ["../model", "../media/audio", "../media/audio-chunks", "../media/documents"]:
        shutil.rmtree(folder)
        os.makedirs(folder)
       
# Create the folders.
for folder in folders:
    create_and_clean_folders(folder)

d:\01. Edwin\01. Projects\Emotion Detection\notebook
d:\01. Edwin\01. Projects\Emotion Detection\notebook
d:\01. Edwin\01. Projects\Emotion Detection\notebook
d:\01. Edwin\01. Projects\Emotion Detection\notebook
d:\01. Edwin\01. Projects\Emotion Detection\notebook
d:\01. Edwin\01. Projects\Emotion Detection\notebook
d:\01. Edwin\01. Projects\Emotion Detection\notebook


### **I. Video Facial - Emotion Detection**

#### Analyzing Video using DeepFace library.

I am going to use the `DeepFace` library to do facial emotion detection for video frames.

Here is the process:

1. Open the video specified using `OpenCV`.
Use `DeepFace` and predict the emotion for each frame of the video.
3. Record the frame's video time and the emotion to a file.
4. Read the file and find the predominant emotion for each second.
5. Create another file specifying the video time (in second) and the predominant emotion.

In [6]:
# Open the video file in the ./media/video folder and assign its path to a variable.
video_folder = "../media/video"
video_file_path = None
video_file_name = None

# Find the video file in the ./media/video folder.
for file in os.listdir(video_folder):
    if file.lower().endswith((".mp4", ".avi", ".mov", ".mkv")):
        video_file_path = os.path.join(video_folder, file)
        video_file_name = file
        break
    
if video_file_path:
    print(f"Video exist at path: {video_file_path}")
else:
    print("No video file found in the ./media/video folder.")

Video exist at path: ../media/video\Woody Roseland - TEDx.mp4


In [7]:
# Files for saving the analysis.
documents_folder = "../media/documents"
frame_analysis_file_header = ["time", "emotion"]
frame_analysis_file = f"{documents_folder}/frame_analysis.csv"
video_analysis_file = f"{documents_folder}/video_analysis.csv"

In [8]:
# Setting variables up for processing.
capture = cv.VideoCapture(video_file_path)
prev_video_time = datetime.timedelta(seconds=0)
window_name = video_file_name
cv.namedWindow(window_name, cv.WINDOW_NORMAL)

# Define the constant variables.
max_threads = 8
frame_skip = 5
frame_id = 0

# Function to handle the emotion detection and face annotation
def process_frame(frame, current_video_time):
    try:
        # Analyzing the emotion in the frame using GPU acceleration if possible
        frame_emotion = DeepFace.analyze(img_path=frame, actions=[
                                         "emotion"], enforce_detection=False, detector_backend='mtcnn')
        emotion = frame_emotion[0].get(
            "dominant_emotion", "No emotion detected")
        
    except Exception as e:
        print(f"Exception: {e}")
    return [current_video_time, emotion]

# Open the CSV file an ouput the analysis for the file.
with open(frame_analysis_file, 'w', newline='') as out_frame_file, ThreadPoolExecutor(max_workers=max_threads) as executor:
    writer = csv.writer(out_frame_file)
    writer.writerow(frame_analysis_file_header)
    
    # Getting the result (video is available or not) and the frame of the capture/video.
    while True:
        ret, frame = capture.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Skip 5 frames
        frame_id += 1
        if frame_id % frame_skip != 0:
            continue

        # Getting the frames and calculating the video duration.
        current_frame_no = capture.get(cv.CAP_PROP_POS_FRAMES)
        total_frame_no = capture.get(cv.CAP_PROP_FRAME_COUNT)
        fps = capture.get(cv.CAP_PROP_FPS)
        
        # Calculate Current Video Time
        current_video_time = datetime.timedelta(seconds=round(current_frame_no/fps))
        
        print(f"Current at Frame: {current_frame_no} of {total_frame_no} with fps {fps}")
        print(f"Current Video Time: {current_video_time}")

        # Submit frame for processing
        future = executor.submit(process_frame, frame, current_video_time)
        result = future.result()
        writer.writerow(result)

        # Break the loop once the video finishes.
        if current_frame_no >= total_frame_no:
            break
        
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    print("Finished writing video time and emotion for all frames of the video.")

capture.release()
cv.destroyAllWindows()

Current at Frame: 5.0 of 1936.0 with fps 23.976023976023978
Current Video Time: 0:00:00
1/1 [==============================] - 0s 28ms/step
Current at Frame: 10.0 of 1936.0 with fps 23.976023976023978
Current Video Time: 0:00:00
1/1 [==============================] - 0s 28ms/step
Current at Frame: 15.0 of 1936.0 with fps 23.976023976023978
Current Video Time: 0:00:01
1/1 [==============================] - 0s 26ms/step
Current at Frame: 20.0 of 1936.0 with fps 23.976023976023978
Current Video Time: 0:00:01
1/1 [==============================] - 0s 25ms/step
Current at Frame: 25.0 of 1936.0 with fps 23.976023976023978
Current Video Time: 0:00:01
1/1 [==============================] - 0s 25ms/step
Current at Frame: 30.0 of 1936.0 with fps 23.976023976023978
Current Video Time: 0:00:01
1/1 [==============================] - 0s 473ms/step
Current at Frame: 35.0 of 1936.0 with fps 23.976023976023978
Current Video Time: 0:00:01
1/1 [==============================] - 0s 28ms/step
Current at Fr